# **P.S. This entire notebook was coded to run on a CPU for compatibility.**

In [1]:
!pip install -q ultralytics timm

# Download files

In [2]:
import gdown

#download model
gdown.download_folder(id='1QVWjX8Aa2VvIbXjDY4DyqcV_Uqw6myX7', output="models", quiet=False, use_cookies=False)

#download test dataset
gdown.download_folder(id='1LZIhiV9l82W4fNpfaBHyGvh8XBX6SoU2', output="test_folder", quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1_10GTTSqYCDbFgEdzRyU3YUXgaGc8aqN labels.txt
Processing file 1wPp6Yx6uqFJH1mCx6I5wHCY2UsRFHecj cnn.pt
Processing file 11sJJF6_W7UbGbskoFSBZ0dMTzkHNCLBe vit.pth
Processing file 1zJraAjjLVcVVxWk9iWzKOgPcL04rqshq README.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1_10GTTSqYCDbFgEdzRyU3YUXgaGc8aqN
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\model\labels.txt
100%|██████████| 124/124 [00:00<?, ?B/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1wPp6Yx6uqFJH1mCx6I5wHCY2UsRFHecj
From (redirected): https://drive.google.com/uc?id=1wPp6Yx6uqFJH1mCx6I5wHCY2UsRFHecj&confirm=t&uuid=8ea27aeb-00b0-4c35-974d-c09067ac2555
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\model\cnn.pt
100%|██████████| 125M/125M [00:05<00:00, 23.8MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=11sJJF6_W7UbGbskoFSBZ0dMTzkHNCLBe
From (redirected): https://drive.google.com/uc?id=11sJJF6_W7UbGbskoFSBZ0dMTzkHNCLBe&confirm=t&uuid=55f6f261-2d4c-43da-9439-786d5ffa5033
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\model\vit.pth
100%|██████████|

Processing file 12ufiwhl6whVp33F05RLiDRieRSN13XlP test_labels.txt
Retrieving folder 1qO-cQtRdp-hLu6LMQ68OEv0zA791puJO test_images
Processing file 1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH Kek Lapis.jpg
Processing file 17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB- Kuih Seri Muka.png
Processing file 1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM Kuih Lapis.jpg
Processing file 1KmI2JRpz4DnCAmPbCWoK31BJBH4ax8i7 Kuih Ubi Kayu.jpg
Processing file 1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4 Kuih Kaswi Pandan.jpg
Processing file 1MEhhjAoBK66wtj6Xs1orsrA2vR9qLjhR Kuih Talam.jpg
Processing file 11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6 Kuih Ketayap.jpg
Processing file 1QFZdtM4ok-vH0kPgA7gdZQzT1aSppWcF Onde-Onde.jpg


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=12ufiwhl6whVp33F05RLiDRieRSN13XlP
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\test_folder\test_labels.txt
100%|██████████| 283/283 [00:00<?, ?B/s] 
Downloading...
From: https://drive.google.com/uc?id=1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\test_folder\test_images\Kek Lapis.jpg
100%|██████████| 1.27M/1.27M [00:00<00:00, 10.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB-
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\test_folder\test_images\Kuih Seri Muka.png
100%|██████████| 1.04M/1.04M [00:00<00:00, 11.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC\naic\test_folder\test_images\Kuih Lapis.jpg
100

['test_folder\\test_labels.txt',
 'test_folder\\test_images\\Kek Lapis.jpg',
 'test_folder\\test_images\\Kuih Seri Muka.png',
 'test_folder\\test_images\\Kuih Lapis.jpg',
 'test_folder\\test_images\\Kuih Ubi Kayu.jpg',
 'test_folder\\test_images\\Kuih Kaswi Pandan.jpg',
 'test_folder\\test_images\\Kuih Talam.jpg',
 'test_folder\\test_images\\Kuih Ketayap.jpg',
 'test_folder\\test_images\\Onde-Onde.jpg']

# Load CNN model

In [3]:
from ultralytics import YOLO

cnn = YOLO("models/cnn.pt")

ERROR Error reading from C:\Users\ochon\AppData\Roaming\Ultralytics\settings.json: "No Ultralytics setting 'openvino_msg'. \nView Ultralytics Settings with 'yolo settings' or at 'C:\\Users\\ochon\\AppData\\Roaming\\Ultralytics\\settings.json'\nUpdate Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings."
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\ochon\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Load ViT

In [4]:
import timm
import torch

vit = timm.create_model('eva02_base_patch14_224.mim_in22k', pretrained=False, num_classes=8)

vit.load_state_dict(torch.load("models/vit.pth", map_location=torch.device('cpu')))

vit.eval().cpu()

c:\Users\ochon\.conda\envs\naic\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Eva(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (rope): RotaryEmbeddingCat()
  (blocks): ModuleList(
    (0-11): 12 x EvaBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): EvaAttention(
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): SwiGLU(
        (fc1_g): Linear(in_features=768, out_features=2048, bias=True)
        (fc1_x): Linear(in_features=76

In [5]:
from torchvision import transforms

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load labels

In [7]:
with open('models/labels.txt') as f:
    labels = {i: line.strip().split(' ', 1)[1] for i, line in enumerate(f)}

print(labels)

{0: 'Kek Lapis', 1: 'Kuih Kaswi Pandan', 2: 'Kuih Ketayap', 3: 'Kuih Lapis', 4: 'Kuih Seri Muka', 5: 'Kuih Talam', 6: 'Kuih Ubi Kayu', 7: 'Onde-Onde'}


# Load test images

In [8]:
import os

test_images = [os.path.join(r, f) for r, d, fs in os.walk('test_folder/test_images') for f in fs if f.lower()]

# CNN predict

In [9]:
from collections import Counter

num_classes = 8
cnn_predictions = []

for path in test_images:
    cnn_results = cnn(path, verbose=False)
    cnn_pred_classes = cnn_results[0].boxes.cls.tolist()  # List of detected class IDs

    if len(cnn_pred_classes) > 0:
        cnn_counts = Counter(cnn_pred_classes)
        cnn_probs = [cnn_counts.get(i, 0) for i in range(num_classes)]
        cnn_probs = [p / sum(cnn_probs) for p in cnn_probs]  # normalize to sum to 1
        cnn_predicted_class_idx = int(max(cnn_counts, key=cnn_counts.get))
    else:
        cnn_probs = [0.0] * num_classes
        cnn_predicted_class_idx = -1

    cnn_predictions.append({
        'image': os.path.basename(path),
        'predicted_class_index': cnn_predicted_class_idx,
        'predicted_label': labels.get(cnn_predicted_class_idx, 'Unknown'),
        'class_probabilities': cnn_probs
    })

# ViT predict

In [10]:
import torch.nn.functional as F
from PIL import Image

vit_predictions = []

for path in test_images:
    image = Image.open(path).convert("RGB")
    input_tensor = vit_transform(image).unsqueeze(0).cpu()

    with torch.no_grad():
        vit_logits = vit(input_tensor)

    vit_probs_tensor = F.softmax(vit_logits, dim=1).squeeze(0)
    vit_probs = vit_probs_tensor.tolist()

    vit_predicted_class_idx = int(torch.argmax(vit_probs_tensor).item())

    vit_predictions.append({
        'image': os.path.basename(path),
        'predicted_class_index': vit_predicted_class_idx,
        'predicted_label': labels.get(vit_predicted_class_idx, 'Unknown'),
        'class_probabilities': vit_probs
    })

print(vit_predictions)

[{'image': 'Kek Lapis.jpg', 'predicted_class_index': 0, 'predicted_label': 'Kek Lapis', 'class_probabilities': [0.9998119473457336, 2.198141191911418e-05, 2.3216731278807856e-05, 3.7660367524949834e-05, 2.7468486223369837e-05, 2.642873369040899e-05, 2.8511405616882257e-05, 2.286418020958081e-05]}, {'image': 'Kuih Kaswi Pandan.jpg', 'predicted_class_index': 1, 'predicted_label': 'Kuih Kaswi Pandan', 'class_probabilities': [2.2194763005245477e-05, 0.9998972415924072, 1.859329677245114e-05, 2.0087854863959365e-05, 1.404506474500522e-05, 1.1336959687469061e-05, 1.0026237760030199e-05, 6.448821295634843e-06]}, {'image': 'Kuih Ketayap.jpg', 'predicted_class_index': 2, 'predicted_label': 'Kuih Ketayap', 'class_probabilities': [1.139002779382281e-05, 1.3869735994376242e-05, 0.9999200105667114, 9.086345016839914e-06, 1.0420496437291149e-05, 1.1227074537600856e-05, 9.81194716587197e-06, 1.4303759598988108e-05]}, {'image': 'Kuih Lapis.jpg', 'predicted_class_index': 3, 'predicted_label': 'Kuih Lap

# Combine model outputs and decide on one class

In [11]:
final_predictions = []

for cnn, vit in zip(cnn_predictions, vit_predictions):
    assert cnn['image'] == vit['image'], "Mismatch in image order"

    cnn_idx = cnn['predicted_class_index']
    vit_idx = vit['predicted_class_index']
    cnn_probs = cnn['class_probabilities']
    vit_probs = vit['class_probabilities']

    #average probabilities (soft voting)
    final_probs = [(c + v) / 2 for c, v in zip(cnn_probs, vit_probs)]

    # Decide class index from final_probs
    if cnn_idx == vit_idx and cnn_idx != -1: #models cohere
        final_idx = cnn_idx

    elif cnn_idx == -1 and vit_idx != -1: #CNN no output, ViT has output, follow ViT
        final_idx = vit_idx

    elif vit_idx == -1 and cnn_idx != -1: #ViT no output, CNN has output, follow CNN
        final_idx = cnn_idx

    elif cnn_idx != -1 and vit_idx != -1: #models disagree, compare confidences from both models using soft voting
        final_idx = int(torch.tensor(final_probs).argmax().item())
    else:
        final_idx = -1  #both models no output

    final_predictions.append({
        'image': cnn['image'],
        'predicted_class_index': final_idx,
        'predicted_label': labels.get(final_idx, 'Unknown'),
        'class_probabilities': final_probs
    })

# Save results to CSV

In [12]:
import pandas as pd

final_results_df = pd.DataFrame(final_predictions)
print(final_results_df)
final_results_df.to_csv("predictions.csv", index=False)

                   image  predicted_class_index    predicted_label  \
0          Kek Lapis.jpg                      0          Kek Lapis   
1  Kuih Kaswi Pandan.jpg                      1  Kuih Kaswi Pandan   
2       Kuih Ketayap.jpg                      2       Kuih Ketayap   
3         Kuih Lapis.jpg                      3         Kuih Lapis   
4     Kuih Seri Muka.png                      4     Kuih Seri Muka   
5         Kuih Talam.jpg                      5         Kuih Talam   
6      Kuih Ubi Kayu.jpg                      6      Kuih Ubi Kayu   
7          Onde-Onde.jpg                      7          Onde-Onde   

                                 class_probabilities  
0  [0.9999059736728668, 1.099070595955709e-05, 1....  
1  [1.1097381502622738e-05, 0.9999486207962036, 9...  
2  [5.695013896911405e-06, 6.934867997188121e-06,...  
3  [2.9208389605628327e-05, 1.5045891814224888e-0...  
4  [7.1605018092668615e-06, 8.081198757281527e-06...  
5  [6.583813956240192e-06, 6.0684060372

# Map the true labels of the images in the testing folder to model's class index (from here onwards will be the original organisers' code)

In [13]:
df_test = pd.read_csv('test_folder/test_labels.txt')

filename_to_class = dict(zip(df_test['Filename'].str.strip(), df_test['Class'].str.strip()))

label_to_class_index = {v.strip(): k for k, v in labels.items()}

true_class_index = [label_to_class_index[filename_to_class[os.path.basename(path).strip()]] for path in test_images]

In [14]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import label_binarize
import numpy as np

final_results_df['true_class_index'] = true_class_index  ## changes here in v2 - refer to true_class_index
y_true = final_results_df['true_class_index'].astype(int).values
y_pred = final_results_df['predicted_class_index'].astype(int).values
y_probs = np.array(final_results_df['class_probabilities'].tolist())

n_classes=8

# Pad probability vectors to length 8
def pad_probs(probs, target_len=n_classes):
    padded = np.zeros(target_len)
    padded[:len(probs)] = probs  # assumes probs are in order (class 0, 1, 2, ...)
    return padded

# Apply padding
y_probs_padded = np.array([pad_probs(p, n_classes) for p in final_results_df['class_probabilities']])

# Update your DataFrame or use directly in metrics
y_probs = y_probs_padded


In [15]:
# Number of classes
class_names = list(range(n_classes))

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")

# Precision, Recall, F1 per class & macro
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_names, average=None)
macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

print("\n📊 Per-class metrics:")
for i, cls in enumerate(class_names):
    print(f"Class {cls}: Precision={prec[i]:.4f}, Recall={rec[i]:.4f}, F1={f1[i]:.4f}")

print(f"\n📦 Macro Precision: {macro_prec:.4f}, Macro Recall: {macro_rec:.4f}, Macro F1: {macro_f1:.4f}")

# ROC AUC (requires binarized labels)
y_true_bin = label_binarize(y_true, classes=class_names)

# ROC AUC per class and macro
try:
    auc_per_class = roc_auc_score(y_true_bin, y_probs, average=None, multi_class='ovr')
    auc_macro = roc_auc_score(y_true_bin, y_probs, average='macro', multi_class='ovr')

    print("\n🎯 ROC AUC per class:")
    for i, cls in enumerate(class_names):
        print(f"Class {cls}: AUC = {auc_per_class[i]:.4f}")

    print(f"\n🌐 Macro ROC AUC: {auc_macro:.4f}")

except Exception as e:
    print(f"⚠️ ROC AUC could not be computed: {e}")



✅ Accuracy: 1.0000

📊 Per-class metrics:
Class 0: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 1: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 2: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 3: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 4: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 5: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 6: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 7: Precision=1.0000, Recall=1.0000, F1=1.0000

📦 Macro Precision: 1.0000, Macro Recall: 1.0000, Macro F1: 1.0000

🎯 ROC AUC per class:
Class 0: AUC = 1.0000
Class 1: AUC = 1.0000
Class 2: AUC = 1.0000
Class 3: AUC = 1.0000
Class 4: AUC = 1.0000
Class 5: AUC = 1.0000
Class 6: AUC = 1.0000
Class 7: AUC = 1.0000

🌐 Macro ROC AUC: 1.0000
